In [ ]:
import logging
import os
from dotenv import load_dotenv
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import uuid
load_dotenv()
dbpassword = os.getenv('POSTGRES_PASS')
dbuser = os.getenv('POSTGRES_USER')
host_ip = os.getenv('HOST')

In [162]:
df_clans = pd.read_csv('data/clan_sample_data.csv')

In [163]:
def connect_to_db():
    
    try:
        connection = psycopg2.connect(
        host=host_ip,
        # host="127.0.0.1",
        port="5432",
        database="vertigodb",
        user=dbuser,
        password=dbpassword)
        logging.info("Database connection successful.")
        print("Database connection successful.")
        return connection

    except Exception as e:
        logging.error(f"Database connection failed:{e}")
        print(f"Database connection failed: {e}")
        return None


def column_filterings(df):
    # Date column filtering
    df['created_at'] = df['created_at'].astype(str)
    date_regex = r'^\d{4}-\d{2}-\d{2}'
    df = df[df['created_at'].str.match(date_regex)]
    
    # Region column filtering
    df['region'] = df['region'].astype(str)
    # Sadece 2 harfli (ör: 'EU', 'TR') olanları al
    region_regex = r'^[A-Za-z]{2}$'
    df = df[df['region'].str.match(region_regex)]
    return df


def execute_values(conn, df, table):
    df = df.where(pd.notnull(df), None)
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [164]:
df_clans = column_filterings(df_clans)

/var/folders/42/rntxlmpd2wnczlr1hn7mzk6r0000gn/T/ipykernel_25184/921388930.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['region'] = df['region'].astype(str)


In [168]:
connection = connect_to_db()
cursor = connection.cursor()
create_clans = '''CREATE TABLE IF NOT EXISTS clans(id uuid DEFAULT gen_random_uuid(), name char(20) ,region char(5), created_at TIMESTAMP, PRIMARY KEY (id));'''
cursor.execute(create_clans)
sql_trunc_clans = """ TRUNCATE TABLE clans """
cursor.execute(sql_trunc_clans)
execute_values(connection, df_clans, 'clans') 
connection.commit()
cursor.close()

Database connection successful.
execute_values() done


In [ ]:
def get_clan_id(name, region):
    connection = connect_to_db()
    cursor = connection.cursor()
    create_clans = '''SELECT id FROM clans WHERE name = %s AND region = %s;'''
    cursor.execute(create_clans, (name, region)) 
    result = cursor.fetchone()
    cursor.close()
    connection.close()
    return result[0]

In [178]:
get_clan_id('ali', 'TR')

Database connection successful.


'8fea4d89-5283-4864-8f73-190457aeb118'

In [179]:

connection = connect_to_db()
cursor = connection.cursor()
select_query = "SELECT DISTINCT(name) FROM clans"
cursor.execute(select_query)
clan_list = cursor.fetchall()


Database connection successful.


In [193]:
clan_list[0]

('EchoReign           ',)

In [2]:
len([1]) == 0

False

In [191]:
clan_list_stripped = [row[0].strip() for row in clan_list]

In [192]:
clan_list_stripped

['EchoReign',
 'NightRaiders',
 'AshenFury',
 'BlightClaw',
 'SpecterSquad',
 'PulseCore',
 'DarkVanguard',
 'HexBlades',
 'BoneLegion',
 'StormPhantom',
 'CyberRebels',
 'CrimsonFangs',
 'ObsidianCircle',
 'FireBrand',
 'WitchHunters',
 'EmberScythe',
 'VortexHunters',
 'GhostProtocol',
 'SteelWolves',
 'PhantomRise',
 'AbyssOrder',
 'IronVultures',
 'BloodHawks',
 'CrimsonOrder',
 'Darklight',
 'ShadowReavers',
 'NightWatch',
 'NullSector',
 'ZenithGuard',
 'DeadZone',
 'NovaSpire',
 'SilentStorm',
 'GraveSentinels',
 'PhantomCore',
 'Dustborn',
 'IceDrifters',
 'DoomWardens',
 'NovaRiders',
 'IronClaw',
 'VoidClan',
 'FrostShade',
 'AshCoven',
 'Voidborn',
 'StormLegion',
 'WraithUnit',
 'TwilightCoven']